# C1 W1 Group 8

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import seaborn as sns

from src.data import GT_QSD1_W1_LIST
from src.paths import BBDD_PATH, QSD1_W1_PATH
from src.descriptors import GreyScaleHistogramDescriptor1D, HSVHistogramDescriptor1D, RGBHistogramDescriptor1D
from src.similarities import MSE, L1Distance, ChiSquaredDistance, HistogramIntersection, HellingerKernel

## Task 1 - Create Museum and query image descriptors (BBDD & QSD1)

The functions in this section are required to take a PIL.Image object as input and return a 1D descriptor in the form of a NumPy array. Inside the function, you have the freedom to implement any processing or transformation steps as long as the input and output types are respected. Specifically:

    Input: A PIL.Image object, which can be manipulated or processed as needed.
    Output: A 1D descriptor, represented as a NumPy array, which could be a histogram, feature vector, or any other type of descriptor derived from the input image.

In [ ]:
# Replace with the function you want to use to generate the descriptors

descriptors_classes = [
    GreyScaleHistogramDescriptor1D(),
    RGBHistogramDescriptor1D(),
    HSVHistogramDescriptor1D()
]

In [ ]:
# Database image descriptors

database_image_PIL_list = [Image.open(db_img_path) for db_img_path in BBDD_PATH.glob("*.jpg")]  # Load once
database_image_descriptors_list = [np.array([descriptor.compute(database_image_PIL) for database_image_PIL in database_image_PIL_list]) for descriptor in descriptors_classes]

In [ ]:
# Query image descriptors

query_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_W1_PATH.glob("*.jpg")]  # Load once
query_image_descriptors_list = [np.array([descriptor.compute(query_image_PIL) for query_image_PIL in query_image_PIL_list]) for descriptor in descriptors_classes]

In [ ]:
for idx, query_image_descriptors in enumerate(query_image_descriptors_list):
    bins = np.arange(query_image_descriptors.shape[1])
    plt.figure(figsize=(10, 6))
    for i, hist in enumerate(query_image_descriptors):
        plt.plot(bins, hist, label=f'Image {i}')
    plt.title(f'Descriptor {descriptors_classes[idx].__class__.__name__}')
    plt.xlabel('Bins')
    plt.ylabel('Frequency')
    plt.legend(fontsize=6)
    plt.grid(True)
    plt.show()

## Task 2 - Implement / compute similarity measures to compare images

In this section, the functions should implement various distance measures between query descriptors and database descriptors. The input for both the query and database will be a 2D NumPy array, where each row represents the descriptor of one image. Specifically:

    Query descriptors will have shape (N, K), where N is the number of query images, and K is the length of each descriptor.
    Database descriptors will have shape (M, K), where M is the number of database images, and K is the length of each descriptor.

The output should be a 2D array of shape (N, M), where each entry (i, j) represents the distance between query descriptor N_i and database descriptor M_j.

In [ ]:
similarity_classes = [
    MSE(),
    L1Distance(),
    ChiSquaredDistance(),
    HistogramIntersection(),
    HellingerKernel(),
]

In [ ]:
query_descriptor_distances_to_bbdd_list = []
for query_descriptors, database_descriptors in zip(query_image_descriptors_list, database_image_descriptors_list):
    query_distances_to_bbdd = []
    for similarity_function in similarity_classes:
        query_distances_to_bbdd.append(similarity_function.compute(query_descriptors, database_descriptors))
    query_descriptor_distances_to_bbdd_list.append(query_distances_to_bbdd)

In [ ]:
# Define the number of rows and columns based on descriptors and similarity classes
n_rows = len(descriptors_classes)
n_cols = len(similarity_classes)

# Create a figure with subplots for each descriptor-similarity pair
fig, axes = plt.subplots(n_rows, n_cols, figsize=(35, 10))  # Adjust figsize as needed for your plot

# Iterate through each descriptor and similarity function
for i, descriptor in enumerate(descriptors_classes):
    for j, similarity in enumerate(similarity_classes):
        query_distances_to_bbdd = query_descriptor_distances_to_bbdd_list[i][j]
        
        # Normalize the distances
        normalized_query_distances = (query_distances_to_bbdd - query_distances_to_bbdd.min(axis=0)) / (query_distances_to_bbdd.max(axis=0) - query_distances_to_bbdd.min(axis=0))
        
        # Select the subplot axes for the current descriptor-similarity pair
        ax = axes[i, j]
        
        # Use seaborn heatmap to display the distance matrix
        sns.heatmap(normalized_query_distances, cmap="RdBu_r", cbar=(j == n_cols - 1), ax=ax)
        
        # Add labels and title
        ax.set_xlabel('Database Images')
        ax.set_ylabel('Query Images')
        ax.set_title(f'{descriptor.__class__.__name__} {similarity.__class__.__name__}')
        
# Adjust the layout to prevent overlapping labels
plt.tight_layout()

# Show the entire figure
plt.show()

## Task 3 - Implement retrieval system (retrieve top K results)

In [ ]:
def get_topk_distances(query_distances_to_bbdd: np.array, k: int = 1) -> tuple[list[list], list[list]]:
    # Get the indices of the top k minimum values for each row
    retrieved_bbdd_indices = np.argsort(query_distances_to_bbdd, axis=1)[:, :k]
    
    # Gather the top k scores using the indices
    retrieved_bbdd_similarity = np.take_along_axis(query_distances_to_bbdd, retrieved_bbdd_indices, axis=1)
    
    return retrieved_bbdd_indices.tolist(), retrieved_bbdd_similarity.tolist()

In [ ]:
# Select number of results to be retrieved
k = 1

In [ ]:
retrieved_bbdd_indices_list = []
retrieved_bbdd_similarity_list = []
for query_descriptor_distances_to_bbdd in query_descriptor_distances_to_bbdd_list:
    retrieved_bbdd_indices_tmp = []
    retrieved_bbdd_similarity_tmp = []
    for query_distances_to_bbdd in query_descriptor_distances_to_bbdd:
        retrieved_bbdd_indices, retrieved_bbdd_similarity = get_topk_distances(query_distances_to_bbdd, k=k)
        retrieved_bbdd_indices_tmp.append(retrieved_bbdd_indices)
        retrieved_bbdd_similarity_tmp.append(retrieved_bbdd_similarity)
    retrieved_bbdd_indices_list.append(retrieved_bbdd_indices_tmp)
    retrieved_bbdd_similarity_list.append(retrieved_bbdd_similarity_tmp)

In [ ]:
for descriptor, retrieved_bbdd_indices, retrieved_bbdd_similarity in zip(descriptors_classes, retrieved_bbdd_indices_list, retrieved_bbdd_similarity_list):
    for similarity, retrieved_similarity_bbdd_indices, retrieved_similarity_bbdd_similarity in zip(similarity_classes, retrieved_bbdd_indices, retrieved_bbdd_similarity):
        print(descriptor.__class__.__name__, similarity.__class__.__name__, retrieved_similarity_bbdd_indices, retrieved_similarity_bbdd_similarity, "\n")

In [ ]:
# Metrics



In [ ]:
assert False, "Fix the following code"

In [ ]:
# Visualization

# Plot the images in a subplot
n = len(GT_QSD1_W1_LIST)  # Number of rows (ground truth images)

# Create a figure with n rows and k+1 columns (for the GT and N pred images)
fig, axs = plt.subplots(n, k + 2, figsize=(3 * (k + 2), 3 * n))

# Loop through each ground truth and predicted images
for i in range(n):
    # Load ground truth image
    query_image_path = QSD1_W1_PATH / f"{str(i).zfill(5)}.jpg"
    query_image_PIL = Image.open(query_image_path)

    # Plot ground truth image in the first column
    axs[i, 0].imshow(query_image_PIL)
    axs[i, 0].set_title(f"QUERY - {query_image_path.name}")
    axs[i, 0].axis('off')

    # Load ground truth image
    gt_idx = GT_QSD1_W1_LIST[i][0]  # only one index in GT
    database_image_path = BBDD_PATH / f"bbdd_{str(gt_idx).zfill(5)}.jpg"
    database_image_PIL = Image.open(database_image_path)

    # Plot ground truth image in the first column
    axs[i, 1].imshow(database_image_PIL)
    axs[i, 1].set_title(f"BBDD: {database_image_path.name}")
    axs[i, 1].axis('off')

    # Loop through the retrieved images
    for j in range(k):
        retrieved_idx = indices[i][j]  # 'k' indices retrieved
        retrieved_image_path = BBDD_PATH / f"bbdd_{str(retrieved_idx).zfill(5)}.jpg"
        retrieved_image_PIL = Image.open(retrieved_image_path)
        axs[i, j + 2].imshow(retrieved_image_PIL)
        axs[i, j + 2].set_title(f"PRED {j} - {retrieved_image_path.name}")
        axs[i, j + 2].axis('off')

# Adjust layout
plt.tight_layout()
plt.show()

## Task 4 - Create predictions for blind challenge (QST1)